In [1]:
# Load some libraries for reading the data from the database and plotting 

import django
import sys, os
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.tasks import *

from django.db.models import Count, Sum

p = Project.objects.get(pk=106)

## Stopwords from queries

In [14]:
qs = p.query_set.all()
cats = p.category_set.all()

extra_words = set()
for c in cats:
    for w in c.description.split():
        extra_words = extra_words | set([w.lower()])
    for w in c.name.split():
        extra_words = extra_words | set([w.lower()])
    for q in c.query_set.all():
        print(q.text)
    
print(extra_words)

TS=(("Enhanced oil recovery" OR EOR) AND ("carbon capture and utili?ation" OR CCU OR "climate change mitigation" OR "emissions reduction" OR "negative emissions" OR ("carbon dioxide" OR CO2 AND (capture OR removal OR re-use OR transformation OR usage))))
TS=(((methane OR CH4) NEAR (production OR synthesis)) AND ("carbon capture and utili?ation" OR CCU OR "climate change mitigation" OR "emissions reduction" OR "negative emissions" OR (("carbon dioxide" OR CO2) AND (capture OR removal OR re-use OR transformation OR usage))))
TS=(((minerali?ation OR "mineral carbonation") NOT (cement OR concrete OR desalination OR urea) AND ("carbon sequestration" OR "carbon capture and utili?ation" OR CCU OR "climate change mitigation" OR "emissions reduction" OR "negative emissions" OR (("carbon dioxide" OR CO2) AND (sequestration OR capture OR removal OR re-use OR transformation OR usage)))))
TS=(("Enhanced gas recovery" OR EGR OR "coal bed methane" OR CBM OR "coal seam gas" OR CSG) AND ("carbon captur

In [15]:
f"with stopwords {extra_words}"

"with stopwords {'bioplastics', 'carbonates', 'in', 'nanotube', 'ether', 'general', '(no', 'polyoxymethylene', 'synthetic', 'products', 'h2,', 'polymers', 'fiber', 'eor', 'and', 'coal', 'weatehring', 'seam', 'polymers,', 'recovery,', 'synthesis', 'biofuel', 'carbon', 'weathering', 'chemical', 'fullerene', '(not', 'mineralisation', 'co', 'methane', 'egr/cbm', 'ome', 'weathering)', 'not', 'ethers', '+', 'from', '...', 'ccu', 'formic', 'carbonates)', 'dme', 'bed', 'oil', 'graphene,', 'chemicals', 'different', 'of', 'acid', 'gas', 'enhanced', 'acetic', 'graphene', 'dimethyl', 'production', 'cement/concrete', 'ocean', 'dimethil', 'as', 'water', 'algae', 'other', 'desalination', 'methanol', 'c', 'fiber,', 'urea', 'recovery'}"

In [17]:
q = Query.objects.get(pk=6230)
for method in ["NM","LD"]:
    for K in [10,15,20,25,30]:
        for alpha in [0.01,0.05,0.1]:
            if method=="LD":
                alpha = alpha*10  
            model, created = RunStats.objects.get_or_create(
                method=method,
                min_freq=10,
                fancy_tokenization=True,
                K=K,
                alpha=alpha,
                query=q,
                extra_stopwords=list(extra_words),
                notes = f"with stopwords {extra_words}"
            )
            if created or model.status==0:
                do_nmf.delay(model.pk)